# Reading and processing NeXus choppers

This guide shows how to extract the relevant data from a NeXus representation of a chopper and create a [scippneutron.chopper.DiskChopper](../../generated/modules/scippneutron.chopper.disk_chopper.DiskChopper.rst) object.

We also demonstrate some utilities that Scippneutron provides to process and visualize the data inside NeXus choppers.

In [ ]:
import scipp as sc

## NeXus chopper data

Here, we use fake data which roughly represents what a real chopper loaded from NeXus looks like.
ScippNeutron has a function for generating this data:

In [ ]:
from scippneutron.data import chopper_mockup

chopper = chopper_mockup()
chopper

We can see that there is some information about the slits and geometry of the chopper as well as some timing-related data.
Take a look at the [NXdisk_chopper](https://manual.nexusformat.org/classes/base_classes/NXdisk_chopper.html) documentation for an overview of the fields.

In this case, there already is a `position`.
This typically needs to be computed first, see [scippnexus.compute_positions](https://scipp.github.io/scippnexus/generated/functions/scippnexus.compute_positions.html).

Some fields are nested data groups which happens when a NeXus file contains `NXlog`s.
We can extract the relevant arrays from them using [extract_chopper_from_nexus](../../generated/modules/scippneutron.chopper.nexus_chopper.extract_chopper_from_nexus.rst):

In [ ]:
from scippneutron.chopper import extract_chopper_from_nexus

chopper = extract_chopper_from_nexus(chopper)
chopper

## Converting to a `DiskChopper`

We can assemble all data into a [scippneutron.chopper.DiskChopper](../../generated/modules/scippneutron.chopper.disk_chopper.DiskChopper.rst) object,
which enables better exploration/visualization of the chopper properties.

In [ ]:
from scippneutron.chopper import DiskChopper

disk_chopper = DiskChopper.from_nexus(chopper)
disk_chopper

Note that the chopper phase $\phi$ was derived from the rotation frequency $f$ and the `delay` $\delta_{t}$ using $\phi = 2 \pi f \delta_{t}$.

With the `DiskChopper`, we can for example see at what times the chopper is open and closed:

In [ ]:
pulse_frequency = sc.scalar(14, unit="Hz")

print("Times open:  ", disk_chopper.time_offset_open(pulse_frequency=pulse_frequency))
print("Times closed:", disk_chopper.time_offset_close(pulse_frequency=pulse_frequency))

## Inspecting time-dependent logs

Some of the entries in the raw NeXus data group are time-dependent.
In this section we take a closer look at these logs to gain more insights in how the chopper data is actually recorded.

### Identifying in-phase regions

Frame unwrapping and time-of-flight computation are only feasible when the choppers are in-phase with the neutron source pulses because, otherwise, the wavelength frames vary pulse-by-pulse.

To identify regions where a chopper is in-phase, we can inspect the `rotation_speed` which is the rotation frequency of the chopper.

In [ ]:
rotation_speed = chopper['rotation_speed']
rotation_speed.name = 'rotation_speed'
rotation_speed

The chopper has a long region of near-constant rotation speed surrounded by spin-up and spin-down regions:

In [ ]:
rotation_speed.plot(markersize=2)

The central plateau is the section of the log where the chopper is in-phase with the source (the ESS source has a frequency of 14 Hz).

We use [find_plateaus](../../generated/modules/scippneutron.chopper.filtering.find_plateaus.rst)
and [collapse_plateaus](../../generated/modules/scippneutron.chopper.filtering.collapse_plateaus.rst) to find one or more plateaus in the log data.

Note the `atol` and `min_n_points` parameters, they need to be tuned for the specific input data.

<div class="alert alert-warning">

**Warning**

``find_plateaus`` can potentially falsely identify regions with a small but steady slope as a plateau.
See the function's documentation for details.

</div>

In [ ]:
from scippneutron.chopper import collapse_plateaus, find_plateaus

plateaus = find_plateaus(
    rotation_speed, atol=sc.scalar(1e-3, unit='Hz / s'), min_n_points=10
)
plateaus = collapse_plateaus(plateaus)
plateaus

`find_plateaus` found two plateaus that we can plot with the following helper function:

In [ ]:
def plot_plateaus(raw_data: sc.DataArray, plateaus: sc.DataArray) -> None:
    to_plot = sc.DataGroup({'Rotation Speed': raw_data})
    for plateau in plateaus:
        i = plateau.coords['plateau'].value
        to_plot[f'Plateau {i}'] = sc.DataArray(
            plateau.data.broadcast(dims=['time'], shape=[2]),
            coords={'time': plateau.coords['time']},
        )
    return to_plot.plot(
        ls={f'Plateau {i}': '-' for i in range(len(plateaus))},
        marker={f'Plateau {i}': '|' for i in range(len(plateaus))},
        markersize={'Rotation Speed': 2},
    )

In [ ]:
plot_plateaus(rotation_speed, plateaus)

In this case, the source has a frequency of 14Hz which means that plateau 0 is in phase.
But plateau 1 is not, it is a short region where the chopper slowed down before fully stopping.

We can use [filter_in_phase](../../generated/modules/scippneutron.chopper.filtering.filter_in_phase.rst) to remove all out-of-phase plateaus:

In [ ]:
pulse_frequency = sc.scalar(14.0, unit='Hz')

In [ ]:
from scippneutron.chopper import filter_in_phase

frequency_in_phase = filter_in_phase(
    plateaus, reference=pulse_frequency, rtol=sc.scalar(1e-3)
)
frequency_in_phase

In [ ]:
plot_plateaus(rotation_speed, frequency_in_phase)

Since there is only one plateau left, we can simply index into it to get the chopper frequency:

In [ ]:
frequency = frequency_in_phase['plateau', 0]
frequency

### Inspecting TDC timestamps

The top-dead-center (TDC) timestamps are created every time a marker placed on the chopper disk passes in front of a sensor mounted on the chopper module.
It is essentially reporting at what times the chopper completed a full rotation.

In [ ]:
tdc = chopper['top_dead_center']
tdc

The initial speed-up and late slow-down are visible when simply plotting all the TDC timestamps:

In [ ]:
tdc.plot()

We can filter out the TDCs for the time when the chopper was in-phase by using the time range of the plateau we extracted above:

In [ ]:
low = frequency.coords['time'][0]
high = frequency.coords['time'][1]
tdc_in_phase = tdc[(tdc > low) & (tdc < high)]
tdc_in_phase

We can check that the rate at which the TDC triggers is indeed close to 14Hz.

In [ ]:
diff = tdc_in_phase[1:] - tdc_in_phase[:-1]
rate = 1 / diff.to(unit='s', dtype='float64')
rate.min(), rate.max()

### Computing chopper phase

When constructing the `DiskChopper` at the start of this notebook,
the phase was derived from a single `delay` value and the rotation speed setpoint.

It is however sometimes useful for debugging to compute the actual (time-dependent) phase of the chopper from the TDC information.

The phase can be defined as $\phi = \omega (TDC - T_0)$, where $\omega$ is the angular frequency of the chopper, $TDC$ is a TDC timestamp, and $T_0$ a pulse time.

We already determined the TDC timestamps above.
In practice, we would get $T_0$ from the input NeXus file, but here, we simply make one up:

In [ ]:
pulse_time = sc.datetime('2023-01-19T08:12:03.484012915', unit='ns')

<div class="alert alert-info">

**Note**

The pulse time is typically an array of timestamps and it can be difficult to determine which pulse goes with which chopper period.

</div>

In [ ]:
# We multiply by 1 rad to get the proper `rad*Hz` unit in `omega`
omega = 2 * sc.constants.pi * frequency.data * sc.scalar(1, unit='rad')
phase = omega * (tdc - pulse_time)
phase = phase.to(unit='deg') % sc.scalar(360.0, unit='deg')

phase.plot()

We can clearly see the a central region where the phase is almost constant, and close to the value of 153.7&deg; listed by the `DiskChopper` table at the top of the notebook
(= 2.683 rad).

To each side of the central region are areas where the chopper is completely out of phase, during the spin-up and spin-down periods.

We can take a closer look at the data in the 'in-phase' region where we see some jitter around the target value:

In [ ]:
phase = omega * (tdc_in_phase - pulse_time)
phase = phase.to(unit='deg') % sc.scalar(360.0, unit='deg')
phase.plot()